In [1]:
import kagglehub
import torch
torch.cuda.is_available()

True

In [2]:
weights_dir = kagglehub.model_download(
    "google/gemma/pyTorch/2b-it")

In [3]:
import os
tokenizer_path = os.path.join(weights_dir, "tokenizer.model")

ckpt_path = os.path.join(weights_dir, "gemma-2b-it.ckpt")

In [4]:
!git clone https://github.com/google/gemma_pytorch.git

fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.


In [5]:
import sys

sys.path.append("gemma_pytorch")

In [6]:
from gemma_pytorch.gemma.config import get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM

In [7]:
import torch

model_config = get_config_for_2b()
model_config

GemmaConfig(vocab_size=256000, max_position_embeddings=8192, num_hidden_layers=18, num_attention_heads=8, num_key_value_heads=1, hidden_size=2048, intermediate_size=16384, head_dim=256, rms_norm_eps=1e-06, dtype='bfloat16', quant=False, tokenizer='tokenizer/tokenizer.model')

In [8]:
model_config.tokenizer = tokenizer_path

In [9]:
model_config.quant = "quant" in "2b-it"

In [10]:
torch.set_default_dtype(model_config.get_dtype())

In [11]:
device = torch.device("cuda")
device

device(type='cuda')

In [12]:
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)

In [13]:
model = model.to(device)

# INFERENCE

In [14]:
model.generate("Give me a 5 day workout plan, 2 rest days. Targeting chest. Make each workout around 30 mins long",
device = device,
output_len = 100)

'\n\n**Day 1:**\n\n* Chest press - 4 sets of 8-12 reps\n* Dumbbell flyes - 4 sets of 8-12 reps\n* Decline dumbbell flyes - 4 sets of 8-12 reps\n\n**Day 2:**\n\n* Barbell incline chest press - 4 sets of 8-12 reps\n* Dumbbell incline chest press - 4 sets of 8-12 reps\n* Cable'

In [15]:
def inference(days:int=5, target:str="all around", length:str = "1 hour", rep_range = "8 to 10"):
    prompt = f"Generate me a weekly workout plan. "
    if days:
        prompt += f"Use {days} workout days and {7-days} rest days. "
    if target:
        prompt += f"I want to target the {target}. "
    if length:
        prompt += f"Each workout should be {length} long. "
    if rep_range:
        prompt += f"Aim for the {rep_range} rep range."
    return model.generate(
        prompt,
        device = device,
        output_len = 1000,
    )

In [16]:
print(inference())

 

**Monday:**

* Squats: 5 sets of 10-12 reps
* Lunges: 5 sets of 10-12 reps
* Dumbbell bicep curls: 5 sets of 10-12 reps
* Triceps dips: 5 sets of 10-12 reps

**Tuesday:**

* Push-ups: 5 sets of 10-12 reps
* Plank: 5 sets of 30-60 seconds
* Bodyweight overhead press: 5 sets of 10-12 reps
* Lateral raises: 5 sets of 10-12 reps

**Wednesday:**

* Deadlifts: 5 sets of 10-12 reps
* Dead lunges: 5 sets of 10-12 reps
* Triceps pushdowns: 5 sets of 10-12 reps
* High knees: 5 sets of 30-60 seconds

**Thursday:**

* Leg press: 5 sets of 10-12 reps
* Kettlebell swings: 5 sets of 10-12 reps
* Plank variations: 5 sets of 30-60 seconds
* Chest flys: 5 sets of 10-12 reps

**Friday:**

* Rest

**Saturday:**

* Active rest: Go for a hike, swim, or bike ride.

**Sunday:**

* Active rest: Practice yoga, meditation, or massage.

**Notes:**

* You can adjust the weight for these exercises according to your fitness level. If you are new to a workout, start with lighter weights and work your way up gradua